This a UFC webscrapping and dataset manipulation code for all UFC fight data from 1993 until 2020.

In [1]:
# looking at the advanced stats for a single fight 
import requests
from bs4 import BeautifulSoup
import re
import csv
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

fights = requests.get('http://ufcstats.com/event-details/0b5b6876c2a4723f').text
soupo = BeautifulSoup(fights, 'lxml')
data = []
for p in soupo.findAll('p', {"class":"b-fight-details__table-text"}):
    mystr=p.string
    mystr = re.sub(r'[\ \n]{2,}', '', str(mystr))
    if mystr != 'None':
        data.append(mystr)
    print('{}'.format(mystr), end = ' ')

None None None 48 40 0 0 0 0 0 0 None U-DEC 
 5 5:00 None None None 165 186 1 0 0 0 0 0 None S-DEC 
 5 5:00 None None None 12 15 1 0 2 0 1 0 None KO/TKO Punch 2 1:00 None None None 74 16 4 2 0 0 2 0 None U-DEC 
 3 5:00 None None None 40 23 1 2 0 0 2 2 None S-DEC 
 3 5:00 None None None 18 0 0 0 0 0 0 0 None KO/TKO Punches 1 2:02 None None None 15 41 8 0 0 0 3 0 None U-DEC 
 3 5:00 None None None 5 5 2 0 1 0 2 0 None SUB Arm Triangle 1 2:58 None None None 86 53 1 1 1 0 1 0 None SUB Rear Naked Choke 3 2:13 None None None 38 54 0 2 0 0 0 1 None S-DEC 
 3 5:00 None None None 17 11 0 0 0 0 0 0 None KO/TKO Punch 1 3:01 

In [ ]:
# main loop for creating the advanced stats csv file

def get_winner(input_link):
	linko = []
	fight_winner = requests.get(input_link).text
	soup_winner = BeautifulSoup(fight_winner,'lxml')

	for everylink in soup_winner.findAll('div',{"class":"b-fight-details__person"}):
		linky = re.sub(r'[\ \n]{2,}', '', str(everylink.text))

		if linky[0:1] == 'W':
			if '"' in linky:
				num = linky.index('"')
				linko.append(linky[1:num])
			else:
				linko.append(linky[1:])

	#See if its a title fight

	if 'Title Bout' in soup_winner.find('i',{"class":"b-fight-details__fight-title"}).text:
		linko.append('Title Bout')

	else:
		linko.append('Non-Title Bout')

	for img in soup_winner.findAll('img'):		

		if 'http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/fight.png' == img.get('src'):
			linko.append("FIGHT")
		elif 'http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/perf.png' == img.get('src'):
			linko.append("PERF")
		elif 'http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/ko.png' == img.get('src'):
			linko.append("KO")
		elif 'http://1e49bc5171d173577ecd-1323f4090557a33db01577564f60846c.r80.cf1.rackcdn.com/sub.png' == img.get('src'):
			linko.append("SUB")
	if len(linko) == 4:
		linko[2] = linko[2] + " " + linko[3]
		linko.pop()
	
	elif 'Title' in linko[0]:
		linko.insert(0,'Draw')

	elif len(linko) == 2:
		linko.append(" ")
	return(linko)



def getnames(input_link):
	z = 0
	names =[]
	fight_detail_request = requests.get(input_link).text
	soup_details = BeautifulSoup(fight_detail_request,'lxml')
	for link in soup_details.findAll('p', {"class":"b-fight-details__table-text"}):
		if z < 2:
			#link = re.sub(r'[\ \n]{2,}', '', str(link.text)
			name = link.text[1:len(link.text) -2]
			names.append(name)
			z +=1
		else:
			break

	for link in soup_details.title.findAll('h2',{"class":"b-content__title"}):
		names.append(link.text)

	wclass = soup_details.find('div',{"class":"b-fight-details__fight-head"}).text
	wclass = re.sub(r'[\ \n]{2,}', '', str(wclass))
	if 'Title' in wclass:
		weight = wclass[4:wclass.index("Title") -1]
		names.append(weight)
	else:
		names.append(wclass)
	return(names)	

#print(getnames("http://www.ufcstats.com/fight-details/fae99a4089f8abb0"))
def getevents(input_link):
	fight_detail_request = requests.get(input_link).text
	soup_details = BeautifulSoup(fight_detail_request,'lxml')
	mylist = []
	myevent = soup_details.find('h2',{"class":"b-content__title"}).text
	myevent = re.sub(r'[\ \n]{2,}', '', str(myevent))
	mylist.append(myevent)
	for link in soup_details.findAll('li',{"class":"b-list__box-list-item"}):
		mystr = re.sub(r'[\ \n]{2,}', '', str(link.text))
		if 'Location' in mystr:
			mylist.append(mystr[9:])
		elif 'Date' in mystr:
			mylist.append(mystr[5:])
		else:
			mylist.append(mystr[11:])
	return(mylist)

print(getevents("http://www.ufcstats.com/event-details/4956f60b7fa57c1a"))

def getstats(input_link):
	fight_stats = []

	fight_stats.extend(getref(input_link))
	fight_stats.extend(get_winner(input_link))
	fight_link = requests.get(input_link).text
	soup_fight = BeautifulSoup(fight_link, 'lxml')
	for stats in soup_fight.findAll('p', {"class":"b-fight-details__table-text"}):
			mystr=stats.string
			mystr = re.sub(r'[\ \n]{2,}', '', str(mystr))
			fight_stats.append(mystr)

	if fight_stats[1] == '1':
		fight_stats += 70 *['N/A']
		for i in range(51,133):
			fight_stats.insert(i,"N/A")

	elif fight_stats[1] == '2':
		fight_stats += 54 *['N/A']
		for i in range(71,131):
			fight_stats.insert(i,"N/A")

	elif fight_stats[1] == '3':
		fight_stats += 36 *['N/A']
		for i in range(91,131):
			fight_stats.insert(i,"N/A")

	elif fight_stats[1] == '4':
		fight_stats += 16 *['N/A']
		for i in range(110,131):
			fight_stats.insert(i,"N/A")


	return(fight_stats)



#132-53 for round 1 finises


def getref(input_link):
	#txt is the long string that is returned from soup
	txt = []
	#ref_clean is the clean appened list of things we want to use
	ref_clean = []
	fight_link = requests.get(input_link).text
	soup_ref = BeautifulSoup(fight_link, 'lxml')
	for refs in soup_ref.findAll('p', {"class":"b-fight-details__text"}):
		mystr = refs.text
		mystr = re.sub(r'[\ \n]{2,}', '', str(mystr))
		txt.append(mystr) 

	#Parce through the online long string that is fighter details

	Method_num = txt[0].index('Round')
	ref_clean.append(txt[0][7:Method_num])

	Round_num = txt[0].index('Time')
	ref_clean.append(txt[0][Method_num+6:Round_num])

	Rndnum = txt[0].index('Time format')
	ref_clean.append(txt[0][Round_num+5:Rndnum])

	Time_num = txt[0].index('Referee')
	ref_clean.append(txt[0][Rndnum+12:Time_num])

	referee = txt[0][Time_num+8:]
	ref_clean.append(referee)

	ref_clean.append(txt[1][8:])
	return(ref_clean)



#MAIN LOOP
def main():
	#main soup for events page
	source = requests.get('http://ufcstats.com/statistics/events/completed?page=all').text
	soup = BeautifulSoup(source, 'lxml')
	x=0

	with open(r'C:\Users\brock\OneDrive\Documents\quick_join.csv','w', newline='') as file:
		writer = csv.writer(file)
		csvheader = ['FIGHTER_RED','FIGHER_BLUE','WEIGHT','EVENT','DATE','STATE_COUNTRY','ATTENDENCE','FINISH','ROUND_FINISHED','TIME_FINISHED','SCHEDUELED ROUNDS','REFEREE','METHOD_OF_FINISH',
				     'WINNER','TITLE_OR_NOT','BONUS','?','?','KD_RED','KD_BLUE','SIG_STR_RED','SIG_STR_BLUE','SIG_STR_PERCENT_RED','SIG_STR_PERCENT_BLUE',
				     'TOTAL_STR_RED','TOTAL_STR_BLUE','TD_RED','TD_BLUE','TD_PERCENT_RED','TD_PERCENT_BLUE','SUB_ATT_RED','SUB_ATT_BLUE','PASSES_RED',
				     'PASSES_BLUE','REV_RED','REV_BLUE','?','?','RND1_KD_RED','RND1_KD_BLUE','RND1_SIG_STR_RED','RND1_SIG_STR_BLUE','RND1_SIG_STR_PERC_RED',
				     'RND1_SIG_STR_PERC_BLUE','RND1_TOTAL_STR_RED','RND1_TOTAL_STR_BLUE','RND1_TD_RED','RND1_TD_BLUE','RND1_TD_PERC_RED','RND1_TD_PERC_BLUE',
				     'RND1_SUB_ATT_RED','RND1_SUB_ATT_BLUE','RND1_PASSES_RED','RND1_PASSES_BLUE','RND1_REV_RED','RND1_REV_BLUE','?','?','RND2_KD_RED','RND2_KD_BLUE',
				     'RND2_SIG_STR_RED','RND2_SIG_STR_BLUE','RND2_SIG_STR_PERC_RED',
				     'RND2_SIG_STR_PERC_BLUE','RND2_TOTAL_STR_RED','RND2_TOTAL_STR_BLUE','RND2_TD_RED','RND2_TD_BLUE','RND2_TD_PERC_RED','RND2_TD_PERC_BLUE',
				     'RND2_SUB_ATT_RED','RND2_SUB_ATT_BLUE','RND2_PASSES_RED','RND2_PASSES_BLUE','RND2_REV_RED','RND2_REV_BLUE','?','?','RND3_KD_RED','RND3_KD_BLUE',
				     'RND3_SIG_STR_RED','RND3_SIG_STR_BLUE','RND3_SIG_STR_PERC_RED',
				     'RND3_SIG_STR_PERC_BLUE','RND3_TOTAL_STR_RED','RND3_TOTAL_STR_BLUE','RND3_TD_RED','RND3_TD_BLUE','RND3_TD_PERC_RED','RND3_TD_PERC_BLUE',
				     'RND3_SUB_ATT_RED','RND3_SUB_ATT_BLUE','RND3_PASSES_RED','RND3_PASSES_BLUE','RND3_REV_RED','RND3_REV_BLUE','?','?','RND4_KD_RED','RND4_KD_BLUE',
				     'RND4_SIG_STR_RED','RND4_SIG_STR_BLUE','RND4_SIG_STR_PERC_RED',
				     'RND4_SIG_STR_PERC_BLUE','RND4_TOTAL_STR_RED','RND4_TOTAL_STR_BLUE','RND4_TD_RED','RND4_TD_BLUE','RND4_TD_PERC_RED','RND4_TD_PERC_BLUE',
				     'RND4_SUB_ATT_RED','RND4_SUB_ATT_BLUE','RND4_PASSES_RED','RND4_PASSES_BLUE','RND4_REV_RED','RND4_REV_BLUE','?','?','RND5_KD_RED','RND5_KD_BLUE',
				     'RND5_SIG_STR_RED','RND5_SIG_STR_BLUE','RND5_SIG_STR_PERC_RED',
				     'RND5_SIG_STR_PERC_BLUE','RND5_TOTAL_STR_RED','RND5_TOTAL_STR_BLUE','RND5_TD_RED','RND5_TD_BLUE','RND5_TD_PERC_RED','RND5_TD_PERC_BLUE',
				     'RND5_SUB_ATT_RED','RND5_SUB_ATT_BLUE','RND5_PASSES_RED','RND5_PASSES_BLUE','RND5_REV_RED','RND5_REV_BLUE','?','?','SIG_STR_TOTAL_RED','SIG_STR_TOTAL_BLUE',
				     'SIG_STR_PERC_RED','SIG_STR_PERC_BLUE','HEAD_STR_RED','HEAD_STR_BLUE','BODY_STR_RED','BODY_STR_BLUE','LEG_STR_RED','LEG_STR_BLUE','DISTANCE_RED','DISTANCE_BLUE',
				     'CLINCH_RED','CLICH_BLUE','GROUD_RED','GROUND_BLUE','?','?','RND1_SIG_STR_RED','RND1_SIG_STR_BLUE','RND1_SIG_STR_PERC_RED','RND1_SIG_STR_PERC_BLUE','RND1_HEAD_STR_RED',
				     'RND1_HEAD_STR_BLUE','RND1_BODY_STR_RED','RND1_BODY_STR_BLUE','RND1_LEG_STR_RED','RND1_LEG_STR_BLUE','RND1_DISTANCE_RED','RND1_DISTANCE_BLUE',
				     'RND1_CLINCH_RED','RND1_CLINCH_BLUE','RND1_GROUND_RED','RND1_GOUND_BLUE','?','?','RND2_SIG_STR_RED','RND2_SIG_STR_BLUE','RND2_SIG_STR_PERC_RED','RND2_SIG_STR_PERC_BLUE','RND1_HEAD_STR_RED',
				     'RND2_HEAD_STR_BLUE','RND2_BODY_STR_RED','RND2_BODY_STR_BLUE','RND2_LEG_STR_RED','RND2_LEG_STR_BLUE','RND2_DISTANCE_RED','RND2_DISTANCE_BLUE',
				     'RND2_CLINCH_RED','RND2_CLINCH_BLUE','RND2_GROUND_RED','RND2_GOUND_BLUE','?','?','RND3_SIG_STR_RED','RND3_SIG_STR_BLUE','RND3_SIG_STR_PERC_RED','RND3_SIG_STR_PERC_BLUE','RND1_HEAD_STR_RED',
				     'RND3_HEAD_STR_BLUE','RND3_BODY_STR_RED','RND3_BODY_STR_BLUE','RND3_LEG_STR_RED','RND3_LEG_STR_BLUE','RND3_DISTANCE_RED','RND3_DISTANCE_BLUE',
				     'RND3_CLINCH_RED','RND3_CLINCH_BLUE','RND3_GROUND_RED','RND3_GOUND_BLUE','?','?','RND4_SIG_STR_RED','RND4_SIG_STR_BLUE','RND4_SIG_STR_PERC_RED','RND4_SIG_STR_PERC_BLUE','RND1_HEAD_STR_RED',
				     'RND4_HEAD_STR_BLUE','RND4_BODY_STR_RED','RND4_BODY_STR_BLUE','RND4_LEG_STR_RED','RND4_LEG_STR_BLUE','RND4_DISTANCE_RED','RND4_DISTANCE_BLUE',
				     'RND4_CLINCH_RED','RND4_CLINCH_BLUE','RND4_GROUND_RED','RND4_GOUND_BLUE','?','?','RND5_SIG_STR_RED','RND5_SIG_STR_BLUE','RND5_SIG_STR_PERC_RED','RND5_SIG_STR_PERC_BLUE','RND1_HEAD_STR_RED',
				     'RND5_HEAD_STR_BLUE','RND5_BODY_STR_RED','RND5_BODY_STR_BLUE','RND5_LEG_STR_RED','RND5_LEG_STR_BLUE','RND5_DISTANCE_RED','RND5_DISTANCE_BLUE',
				     'RND5_CLINCH_RED','RND5_CLINCH_BLUE','RND5_GROUND_RED','RND5_GOUND_BLUE']


		writer.writerow(csvheader)
		print('writing csv')


	for link in soup.findAll('a', attrs={'href': re.compile("^http://")}):
		print(x)
        #set x to a value to the number of desired events, new events can be concatenated later as events occur

		
		x = x + 1
		if x < 1000:
				#check if its an event details link
			if "event-details" in link.get('href'):
				print(link.get('href'))
				#soup2 is the event details link
				#souping up the event link
				source2 = requests.get(link.get('href')).text
				soup2 = BeautifulSoup(source2, 'lxml')

				csvdata = getevents(link.get('href'))
				#looping through event link to get every fight-details link
				for links in soup2.findAll('a', attrs={'href': re.compile("^http://")}):

					if "fight-details" in links.get('href'):
						source3 = requests.get(links.get('href')).text
						soup3 = BeautifulSoup(source3, 'lxml')
						csvdata = getnames(links.get('href'))
						csvdata.extend(getevents(link.get('href')))
						csvdata.extend(getstats(links.get('href')))

						print(getevents(link.get('href')))

						with open(r'C:\Users\brock\OneDrive\Documents\quick_join.csv','a', newline='') as file:
							writer = csv.writer(file)
							#csv row is the stats list
							writer.writerow(csvdata)
							print('writing csv')
													
							
		else:
			break


main()

In [ ]:
advanced = pd.read_csv(r'C:\Users\brock\OneDrive\Documents\quick_join.csv')

In [ ]:
#importing required CSVs and renaming a few column errors, also setting dataset display widths

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
advanced = pd.read_csv(r'C:\Users\brock\OneDrive\Documents\quick_join.csv')
advanced.rename({'TITLE_OR_NOT': 'METHOD_OF_FINISH', 'METHOD_OF_FINISH': 'WINNER','WINNER':'TITLE_OR_NOT'}, axis=1)

In [ ]:
# All duplicated columns with the bonus as None have column formatting issue, columns are shifted to the right by 1.

p1 = advanced[(advanced.duplicated()) & (advanced['BONUS'] == 'None')].iloc[:,0:16]

In [ ]:
# Grabing the 2nd half of the data and shifting it over 1 column to the left to fix the formatting issue
p2 = advanced[(advanced.duplicated()) & (advanced['BONUS'] == 'None')].iloc[:,16:].shift(1,axis='columns',fill_value = advanced[(advanced.duplicated()) & (advanced['BONUS'] == 'None')]['?.1'])

In [6]:
#join the 2 halfs together
df1 = pd.concat([p1,p2], sort = False, axis = 1)
df1

,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,?,?.1,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,PASSES_RED,PASSES_BLUE,REV_RED,REV_BLUE,?.2,?.3,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_PASSES_RED,RND1_PASSES_BLUE,RND1_REV_RED,RND1_REV_BLUE,?.4,?.5,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_PASSES_RED,RND2_PASSES_BLUE,RND2_REV_RED,RND2_REV_BLUE,?.6,?.7,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_PASSES_RED,RND3_PASSES_BLUE,RND3_REV_RED,RND3_REV_BLUE,?.8,?.9,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_PASSES_RED,RND4_PASSES_BLUE,RND4_REV_RED,RND4_REV_BLUE,?.10,?.11,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_PASSES_RED,RND5_PASSES_BLUE,RND5_REV_RED,RND5_REV_BLUE,?.12,?.13,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,?.14,?.15,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,?.16,?.17,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,?.18,?.19,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,?.20,?.21,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,?.22,?.23,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE
111,Marina Rodriguez,Cynthia Calvillo,Women's Strawweight Bout,UFC Fight Night: Overeem vs. Rozenstruik,"December 07, 2019","Washington, DC, USA","10,816",Decision - Majority,3,5:00,3 Rnd (5-5-5),Fernando Yamasaki,Sal D'amato28 - 28.David Braslow28 - 28.Chris ...,Draw,Non-Title

In [7]:
#grabbing the duplicated values with proper column formatting
p3 = advanced[(advanced.duplicated()) & (advanced['BONUS'] != 'None')]
p3

,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,?,?.1,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,PASSES_RED,PASSES_BLUE,REV_RED,REV_BLUE,?.2,?.3,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_PASSES_RED,RND1_PASSES_BLUE,RND1_REV_RED,RND1_REV_BLUE,?.4,?.5,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_PASSES_RED,RND2_PASSES_BLUE,RND2_REV_RED,RND2_REV_BLUE,?.6,?.7,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_PASSES_RED,RND3_PASSES_BLUE,RND3_REV_RED,RND3_REV_BLUE,?.8,?.9,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_PASSES_RED,RND4_PASSES_BLUE,RND4_REV_RED,RND4_REV_BLUE,?.10,?.11,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_PASSES_RED,RND5_PASSES_BLUE,RND5_REV_RED,RND5_REV_BLUE,?.12,?.13,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,?.14,?.15,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,?.16,?.17,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,?.18,?.19,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,?.20,?.21,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,?.22,?.23,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE
1226,Lando Vannata,Bobby Green,Lightweight Bout,UFC 216: Ferguson vs. Lee,"October 07, 2017","Las Vegas, Nevada, USA","10,638",Decision - Split,3,5:00,3 Rnd (5-5-5),Herb Dean,Derek Cleary28 - 28.Jeff Mullen29 - 27.Marcos ...,Draw,Non-Title Bout,FIGHT,None,None,1.0,0,104 of 232,

In [8]:
# adding them to the complete dataset containing all duplicated values
df1 = pd.concat([df1,p3])
df1

,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,?,?.1,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,PASSES_RED,PASSES_BLUE,REV_RED,REV_BLUE,?.2,?.3,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_PASSES_RED,RND1_PASSES_BLUE,RND1_REV_RED,RND1_REV_BLUE,?.4,?.5,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_PASSES_RED,RND2_PASSES_BLUE,RND2_REV_RED,RND2_REV_BLUE,?.6,?.7,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_PASSES_RED,RND3_PASSES_BLUE,RND3_REV_RED,RND3_REV_BLUE,?.8,?.9,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_PASSES_RED,RND4_PASSES_BLUE,RND4_REV_RED,RND4_REV_BLUE,?.10,?.11,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_PASSES_RED,RND5_PASSES_BLUE,RND5_REV_RED,RND5_REV_BLUE,?.12,?.13,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,?.14,?.15,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,?.16,?.17,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,?.18,?.19,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,?.20,?.21,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,?.22,?.23,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE
111,Marina Rodriguez,Cynthia Calvillo,Women's Strawweight Bout,UFC Fight Night: Overeem vs. Rozenstruik,"December 07, 2019","Washington, DC, USA","10,816",Decision - Majority,3,5:00,3 Rnd (5-5-5),Fernando Yamasaki,Sal D'amato28 - 28.David Braslow28 - 28.Chris ...,Draw,Non-Title

In [9]:
# filling NAs as 0
advanced.fillna(value = 0,inplace = True)

In [10]:
# dropping all duplicated items then adding our corrected dataset 
advanced.drop_duplicates(keep = False,inplace = True)
advanced = pd.concat([advanced , df1])

In [11]:
#Appears to be an issue with some columns being shifted from row 138 and onward (looking to fix this in the main loop in the future)
s1 = advanced[advanced['HEAD_STR_RED'].str.contains('%',na=False)].iloc[:,0:137]
s1

,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,?,?.1,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,PASSES_RED,PASSES_BLUE,REV_RED,REV_BLUE,?.2,?.3,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_PASSES_RED,RND1_PASSES_BLUE,RND1_REV_RED,RND1_REV_BLUE,?.4,?.5,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_PASSES_RED,RND2_PASSES_BLUE,RND2_REV_RED,RND2_REV_BLUE,?.6,?.7,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_PASSES_RED,RND3_PASSES_BLUE,RND3_REV_RED,RND3_REV_BLUE,?.8,?.9,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_PASSES_RED,RND4_PASSES_BLUE,RND4_REV_RED,RND4_REV_BLUE,?.10,?.11,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_PASSES_RED,RND5_PASSES_BLUE,RND5_REV_RED,RND5_REV_BLUE,?.12
5,Sean O'Malley,Jose Quinonez,Bantamweight Bout,UFC 248: Adesanya vs. Romero,"March 07, 2020","Las Vegas, Nevada, USA","15,077",KO/TKO,1,2:02,3 Rnd (5-5-5),Dan Miragliotta,Punches to Head On Ground,Sean O'Malley,Non-Title Bout,,None,None,0,0,18 of 22,0 of 11,81%,0%,18 of 22,0 of 11,0 of 0,0 of 0,0%,0%,0,0,0,0,0,0,None,None,0,0,18 of 22,0 of 11,81%,0%,18 of 22,0 of 11,0 of 0,0 of 0,0%,0%,0,0,0,0,0,0,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Rodolfo Vieira,Saparbeg Safarov,Middleweight Bout,UFC 248: Adesanya vs. Romero,"March 07, 2020","Las Vegas, Nevada, USA","15,077",Submission,1,2:58,3 Rnd (5-5-5),Jason Herzog,Arm Triangle From Mount,Rodolfo Vieira,Non-Title Bout,,None,None,0,0,5 of 10,5 of 10,50%,50%,10 of 15,7 of 12,2 of 2,0 of 0,100%,0%,1,0,2,0,0,0,None,None,0,0,5 of 10,5 of 10,50%,50%,10 of 15,7 of 12,2 of 2,0 of 0,100%,0%,1,0,2,0,0,0,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,Danaa Batgerel,Guido Cannetti,Bantamweight Bout,UFC 248: Adesanya vs. Romero,"March 07, 2020","Las Vegas, Nevada, USA","15,077",KO/TKO,1,3:01,3 Rnd (5-5-5),Chris Tognoni,Punch to Head At Distance,Danaa Batgerel,Non-Title Bout,,None,None,1,0,17 of 33,11 of 23,51%,47%,17 of 33,11 of 23,0 of 0,0 of 0,0%,0%,0,0,0,0,0,0,None,None,1,0,17 of 33,11 of 23,51%,47%,17 of 33,11 of 23,0 of 0,0 of 0,0%,0%,0,0,0,0,0,0,None,None,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12,Felicia Spencer,Zarah Fairn,Women's Featherweight Bout,UFC Fight Night: Benavidez vs. Figueiredo,"February 29, 2020","Norfolk, Virginia, USA","7,098",KO/TKO,1,3:37,3 Rnd (5-5-5),Dan Miragliotta,Punches to Head From Mount,Felicia Spencer,Non-Title Bout,,None,None,0,0,40 of 64,13 of 28,62%,46%,68 of 98,14 of 29,1 of 2,0 of 0,50%,

In [12]:
#shifting the data so the columns are in the proper loations
s2 = advanced[advanced['HEAD_STR_RED'].str.contains('%',na=False)].iloc[:,137:].shift(-2,axis =1 )
s2

,?.13,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,?.14,?.15,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,?.16,?.17,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,?.18,?.19,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,?.20,?.21,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,?.22,?.23,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE
5,0,18 of 22,0 of 11,81%,0%,9 of 11,0 of 7,6 of 7,0 of 1,3 of 4,0 of 3,13 of 17,0 of 11,0 of 0,0 of 0,5 of 5,0 of 0,None,None,18 of 22,0 of 11,81%,0%,9 of 11,0 of 7,6 of 7,0 of 1,3 of 4,0 of 3,13 of 17,0 of 11,0 of 0,0 of 0,5 of 5,0 of 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN
7,0,5 of 10,5 of 10,50%,50%,5 of 10,2 of 7,0 of 0,1 of 1,0 of 0,2 of 2,0 of 3,5 of 10,0 of 0,0 of 0,5 of 7,0 of 0,None,None,5 of 10,5 of 10,50%,50%,5 of 10,2 of 7,0 of 0,1 of 1,0 of 0,2 of 2,0 of 3,5 of 10,0 of 0,0 of 0,5 of 7,0 of 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN
10,0,17 of 33,11 of 23,51%,47%,16 of 29,0 of 7,0 of 2,3 of 6,1 of 2,8 of 10,12 of 27,9 of 21,0 of 1,2 of 2,5 of 5,0 of 0,None,None,17 of 33,11 of 23,51%,47%,16 of 29,0 of 7,0 of 2,3 of 6,1 of 2,8 of 10,12 of 27,9 of 21,0 of 1,2 of 2,5 of 5,0 of 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN
12,0,40 of 64,13 of 28,62%,46%,38 of 61,12 of 27,1 of 1,1 of 1,1 of 2,0 of 0,9 of 24,12 of 26,1 of 1,1 of 2,30 of 39,0 of 0,None,None,40 of 64,13 of 28,62%,46%,38 of 61,12 of 27,1 of 1,1 of 1,1 of 2,0 of 0,9 of 24,12 of 26,1 of 1,1 of 2,30 of 39,0 of 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN
13,0,5 of 11,8 of 12,45%,66%,3 of 8,6 of 10,0 of 1,1 of 1,2 of 2,1 of 1,5 of 11,8 of 12,0 of 0,0 of 0,0 of 0,0 of 0,None,None,5 of 11,8 of 12,45%,66%,3 of 8,6 of 10,0 of 1,1 of 1,2 of 2,1 of 1,5 of 11,8 of 12,0 of 0,0 of 0,0 of 0,0 of 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [13]:
#adding the 2 peices together
df2 = pd.concat([s1,s2], axis = 1)
df2

,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,?,?.1,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,PASSES_RED,PASSES_BLUE,REV_RED,REV_BLUE,?.2,?.3,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_PASSES_RED,RND1_PASSES_BLUE,RND1_REV_RED,RND1_REV_BLUE,?.4,?.5,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_PASSES_RED,RND2_PASSES_BLUE,RND2_REV_RED,RND2_REV_BLUE,?.6,?.7,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_PASSES_RED,RND3_PASSES_BLUE,RND3_REV_RED,RND3_REV_BLUE,?.8,?.9,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_PASSES_RED,RND4_PASSES_BLUE,RND4_REV_RED,RND4_REV_BLUE,?.10,?.11,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_PASSES_RED,RND5_PASSES_BLUE,RND5_REV_RED,RND5_REV_BLUE,?.12,?.13,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,?.14,?.15,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,?.16,?.17,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,?.18,?.19,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,?.20,?.21,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,?.22,?.23,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE
5,Sean O'Malley,Jose Quinonez,Bantamweight Bout,UFC 248: Adesanya vs. Romero,"March 07, 2020","Las Vegas, Nevada, USA","15,077",KO/TKO,1,2:02,3 Rnd (5-5-5),Dan Miragliotta,Punches to Head On Ground,Sean O'Malley,Non-Title Bout,,None,None,0,0,18 of 22,0 of 11,81%,0%,18 of 22,0 o

In [15]:
advanced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5552 entries, 0 to 4864
Columns: 244 entries, FIGHTER_RED to RND5_GOUND_BLUE
dtypes: object(244)
memory usage: 10.4+ MB


In [16]:
# removing all incorrectly formatted columns and then adding our corrected dataset
advanced = advanced[advanced['HEAD_STR_RED'].str.contains('%',na=False) == False]
advanced = pd.concat([advanced,df2])

In [17]:
#Quick test
advanced[advanced['WINNER'] == 'Draw']

,FIGHTER_RED,FIGHER_BLUE,WEIGHT,EVENT,DATE,STATE_COUNTRY,ATTENDENCE,FINISH,ROUND_FINISHED,TIME_FINISHED,SCHEDUELED ROUNDS,REFEREE,METHOD_OF_FINISH,WINNER,TITLE_OR_NOT,BONUS,?,?.1,KD_RED,KD_BLUE,SIG_STR_RED,SIG_STR_BLUE,SIG_STR_PERCENT_RED,SIG_STR_PERCENT_BLUE,TOTAL_STR_RED,TOTAL_STR_BLUE,TD_RED,TD_BLUE,TD_PERCENT_RED,TD_PERCENT_BLUE,SUB_ATT_RED,SUB_ATT_BLUE,PASSES_RED,PASSES_BLUE,REV_RED,REV_BLUE,?.2,?.3,RND1_KD_RED,RND1_KD_BLUE,RND1_SIG_STR_RED,RND1_SIG_STR_BLUE,RND1_SIG_STR_PERC_RED,RND1_SIG_STR_PERC_BLUE,RND1_TOTAL_STR_RED,RND1_TOTAL_STR_BLUE,RND1_TD_RED,RND1_TD_BLUE,RND1_TD_PERC_RED,RND1_TD_PERC_BLUE,RND1_SUB_ATT_RED,RND1_SUB_ATT_BLUE,RND1_PASSES_RED,RND1_PASSES_BLUE,RND1_REV_RED,RND1_REV_BLUE,?.4,?.5,RND2_KD_RED,RND2_KD_BLUE,RND2_SIG_STR_RED,RND2_SIG_STR_BLUE,RND2_SIG_STR_PERC_RED,RND2_SIG_STR_PERC_BLUE,RND2_TOTAL_STR_RED,RND2_TOTAL_STR_BLUE,RND2_TD_RED,RND2_TD_BLUE,RND2_TD_PERC_RED,RND2_TD_PERC_BLUE,RND2_SUB_ATT_RED,RND2_SUB_ATT_BLUE,RND2_PASSES_RED,RND2_PASSES_BLUE,RND2_REV_RED,RND2_REV_BLUE,?.6,?.7,RND3_KD_RED,RND3_KD_BLUE,RND3_SIG_STR_RED,RND3_SIG_STR_BLUE,RND3_SIG_STR_PERC_RED,RND3_SIG_STR_PERC_BLUE,RND3_TOTAL_STR_RED,RND3_TOTAL_STR_BLUE,RND3_TD_RED,RND3_TD_BLUE,RND3_TD_PERC_RED,RND3_TD_PERC_BLUE,RND3_SUB_ATT_RED,RND3_SUB_ATT_BLUE,RND3_PASSES_RED,RND3_PASSES_BLUE,RND3_REV_RED,RND3_REV_BLUE,?.8,?.9,RND4_KD_RED,RND4_KD_BLUE,RND4_SIG_STR_RED,RND4_SIG_STR_BLUE,RND4_SIG_STR_PERC_RED,RND4_SIG_STR_PERC_BLUE,RND4_TOTAL_STR_RED,RND4_TOTAL_STR_BLUE,RND4_TD_RED,RND4_TD_BLUE,RND4_TD_PERC_RED,RND4_TD_PERC_BLUE,RND4_SUB_ATT_RED,RND4_SUB_ATT_BLUE,RND4_PASSES_RED,RND4_PASSES_BLUE,RND4_REV_RED,RND4_REV_BLUE,?.10,?.11,RND5_KD_RED,RND5_KD_BLUE,RND5_SIG_STR_RED,RND5_SIG_STR_BLUE,RND5_SIG_STR_PERC_RED,RND5_SIG_STR_PERC_BLUE,RND5_TOTAL_STR_RED,RND5_TOTAL_STR_BLUE,RND5_TD_RED,RND5_TD_BLUE,RND5_TD_PERC_RED,RND5_TD_PERC_BLUE,RND5_SUB_ATT_RED,RND5_SUB_ATT_BLUE,RND5_PASSES_RED,RND5_PASSES_BLUE,RND5_REV_RED,RND5_REV_BLUE,?.12,?.13,SIG_STR_TOTAL_RED,SIG_STR_TOTAL_BLUE,SIG_STR_PERC_RED,SIG_STR_PERC_BLUE,HEAD_STR_RED,HEAD_STR_BLUE,BODY_STR_RED,BODY_STR_BLUE,LEG_STR_RED,LEG_STR_BLUE,DISTANCE_RED,DISTANCE_BLUE,CLINCH_RED,CLICH_BLUE,GROUD_RED,GROUND_BLUE,?.14,?.15,RND1_SIG_STR_RED.1,RND1_SIG_STR_BLUE.1,RND1_SIG_STR_PERC_RED.1,RND1_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED,RND1_HEAD_STR_BLUE,RND1_BODY_STR_RED,RND1_BODY_STR_BLUE,RND1_LEG_STR_RED,RND1_LEG_STR_BLUE,RND1_DISTANCE_RED,RND1_DISTANCE_BLUE,RND1_CLINCH_RED,RND1_CLINCH_BLUE,RND1_GROUND_RED,RND1_GOUND_BLUE,?.16,?.17,RND2_SIG_STR_RED.1,RND2_SIG_STR_BLUE.1,RND2_SIG_STR_PERC_RED.1,RND2_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.1,RND2_HEAD_STR_BLUE,RND2_BODY_STR_RED,RND2_BODY_STR_BLUE,RND2_LEG_STR_RED,RND2_LEG_STR_BLUE,RND2_DISTANCE_RED,RND2_DISTANCE_BLUE,RND2_CLINCH_RED,RND2_CLINCH_BLUE,RND2_GROUND_RED,RND2_GOUND_BLUE,?.18,?.19,RND3_SIG_STR_RED.1,RND3_SIG_STR_BLUE.1,RND3_SIG_STR_PERC_RED.1,RND3_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.2,RND3_HEAD_STR_BLUE,RND3_BODY_STR_RED,RND3_BODY_STR_BLUE,RND3_LEG_STR_RED,RND3_LEG_STR_BLUE,RND3_DISTANCE_RED,RND3_DISTANCE_BLUE,RND3_CLINCH_RED,RND3_CLINCH_BLUE,RND3_GROUND_RED,RND3_GOUND_BLUE,?.20,?.21,RND4_SIG_STR_RED.1,RND4_SIG_STR_BLUE.1,RND4_SIG_STR_PERC_RED.1,RND4_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.3,RND4_HEAD_STR_BLUE,RND4_BODY_STR_RED,RND4_BODY_STR_BLUE,RND4_LEG_STR_RED,RND4_LEG_STR_BLUE,RND4_DISTANCE_RED,RND4_DISTANCE_BLUE,RND4_CLINCH_RED,RND4_CLINCH_BLUE,RND4_GROUND_RED,RND4_GOUND_BLUE,?.22,?.23,RND5_SIG_STR_RED.1,RND5_SIG_STR_BLUE.1,RND5_SIG_STR_PERC_RED.1,RND5_SIG_STR_PERC_BLUE.1,RND1_HEAD_STR_RED.4,RND5_HEAD_STR_BLUE,RND5_BODY_STR_RED,RND5_BODY_STR_BLUE,RND5_LEG_STR_RED,RND5_LEG_STR_BLUE,RND5_DISTANCE_RED,RND5_DISTANCE_BLUE,RND5_CLINCH_RED,RND5_CLINCH_BLUE,RND5_GROUND_RED,RND5_GOUND_BLUE
111,Marina Rodriguez,Cynthia Calvillo,Women's Strawweight Bout,UFC Fight Night: Overeem vs. Rozenstruik,"December 07, 2019","Washington, DC, USA","10,816",Decision - Majority,3,5:00,3 Rnd (5-5-5),Fernando Yamasaki,Sal D'amato28 - 28.David Braslow28 - 28.Chris ...,Draw,Non-Title

In [18]:
# similar column formation issue, creating corrected versions of the data
t1 = advanced[advanced['GROUND_BLUE'] == 'None'].iloc[:,0:137]
t2 = advanced[advanced['GROUND_BLUE'] == 'None'].iloc[:,137:].shift(1,axis=1)

In [19]:
df3 = pd.concat([t1,t2],axis=1)

In [20]:
advanced = advanced[advanced['GROUND_BLUE'] != 'None']

In [21]:
advanced = pd.concat([advanced,df3])
advanced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5552 entries, 0 to 5414
Columns: 244 entries, FIGHTER_RED to RND5_GOUND_BLUE
dtypes: object(244)
memory usage: 10.4+ MB


In [25]:
# removing unneeded columns
for i in advanced.columns:
    if i[0] == "?" or i[0:3] == 'Unn':
        print(i)
        advanced.drop(i,axis = 1,inplace=True)

?
?.1
?.2
?.3
?.4
?.5
?.6
?.7
?.8
?.9
?.10
?.11
?.12
?.13
?.14
?.15
?.16
?.17
?.18
?.19
?.20
?.21
?.22
?.23


In [26]:
#reseting the index 
advanced.reset_index(inplace = True)

In [ ]:
#creating the loser stat, if the fighter isn't a winner and the fight wasn't a draw, that fighter was a loser
advanced['LOSER'] = 0
count = 0
for i in advanced[['FIGHTER_RED','FIGHER_BLUE','WINNER']].values:
    if i[0] not in i[2]:
        advanced['LOSER'].loc[count] = i[0]
    elif i[1] not in i[2]:
        advanced['LOSER'].loc[count] = i[1]
    else:
        advanced['LOSER'].loc[count] = 'Draw'
        
    print(count)
        
    count += 1

In [29]:
#fill NAs as 0
advanced.fillna(value = 0,inplace = True)

In [30]:
advanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5552 entries, 0 to 5551
Columns: 222 entries, index to LOSER
dtypes: float64(35), int64(1), object(186)
memory usage: 9.4+ MB


In [34]:
#writing to csv
advanced.to_csv(r'C:\Users\brock\OneDrive\Documents\UFC_Fight_Stats2.csv')